# Imports

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Model Load

In [5]:
!unzip "/content/drive/MyDrive/Dirllm/distilled_model.zip" -d "/content/"

Archive:  /content/drive/MyDrive/Dirllm/distilled_model.zip
   creating: /content/distilled/
  inflating: /content/distilled/config.json  
  inflating: /content/distilled/generation_config.json  
  inflating: /content/distilled/model.safetensors  


In [6]:
!unzip "/content/drive/MyDrive/Dirllm/Teacher_dataset.zip" -d "/content/"

Archive:  /content/drive/MyDrive/Dirllm/Teacher_dataset.zip
   creating: /content/Teacher_dataset/
 extracting: /content/Teacher_dataset/dataset_dict.json  
   creating: /content/Teacher_dataset/test/
  inflating: /content/Teacher_dataset/test/cache-073a0a915559c680.arrow  
  inflating: /content/Teacher_dataset/test/data-00000-of-00001.arrow  
  inflating: /content/Teacher_dataset/test/dataset_info.json  
  inflating: /content/Teacher_dataset/test/state.json  
   creating: /content/Teacher_dataset/train/
  inflating: /content/Teacher_dataset/train/cache-584a84910cca543c.arrow  
  inflating: /content/Teacher_dataset/train/data-00000-of-00001.arrow  
  inflating: /content/Teacher_dataset/train/dataset_info.json  
  inflating: /content/Teacher_dataset/train/state.json  
   creating: /content/Teacher_dataset/validation/
  inflating: /content/Teacher_dataset/validation/cache-41ad7b289e7d3c9b.arrow  
  inflating: /content/Teacher_dataset/validation/data-00000-of-00001.arrow  
  inflating: /c

In [7]:
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("distilled")
    finetuned_model = finetuned_model.to('cuda')
    print("Load success")

except:
    print("Load Failure")

Load success


In [ ]:
try:
    dataset = load_from_disk("Teacher_dataset")
    print("Loaded Merged Dataset")
except:
    print("Load Failure")

Loaded Merged Dataset


In [8]:
model_name='t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

# Import test set

In [9]:
#Index for question selection
index = int(input("Type in the index of question to be extracted (1 to 200): "))
# Question from test set using index
question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']
#Show the imported test question
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")

Type in the index of question to be extracted (1 to 200): 7


NameError: name 'dataset' is not defined

# Custom Question

Sample Contexts:

CREATE TABLE rec_mart_sales ("Date of Sale" date, "item name" text, "item price" number, "Customer Name" text)

CREATE TABLE rec_print_sales ("Date of Transaction" date, "No. of print pages" number, "No. of Xerox" copies number, "Student name" text, "Amount paid" number)

In [10]:
#custom input
context = input("Write the SQL for context table creation: ")

Write the SQL for context table creation: CREATE TABLE rec_print_sales ("Date of Transaction" date, "No. of print pages" number, "No. of Xerox" copies number, "Student name" text, "Amount paid" number)


In [11]:
question = input("What is the prompt to be asked?: ")
answer = "Flag"

What is the prompt to be asked?: When did Hari takes 10 pages print?


In [12]:
prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

In [13]:
#input tokenisation
inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cuda')

In [14]:
#output inference
output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

In [15]:
output

'SELECT "Date of Transaction" FROM rec_print_sales WHERE "Date of Transaction" = \'10 pages\''

In [24]:
def generate_sql_queries(natural_query, num_samples):
    input_text = f"Convert to SQL: {natural_query}"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check available device

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)  # Move input_ids to GPU

    finetuned_model.to(device)  # Ensure model is on the same device

    outputs = finetuned_model.generate(
        input_ids,
        max_length=100,
        num_return_sequences=num_samples,
        num_beams=10,  # Beam search for diverse outputs
        early_stopping=True
    )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


In [38]:
for i in generate_sql_queries(output, 5):
  print(i)

SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages'
SELECT "Date of Transaction" FROM rec_print_sales WHERE "Date of Transaction" = '10 pages'
SELECT SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages'
SELECT "Date of Transaction" FROM tables_print_sales WHERE "Date of Transaction" = '10 pages'
SELECT "Date of Transaction" FROM SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages'


In [31]:
def get_query_log_prob(query):
    """Compute log probability for a generated SQL query."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check available device

    input_ids = tokenizer(query, return_tensors="pt").input_ids.to(device)  # Move input_ids to GPU
    finetuned_model.to(device)  # Ensure model is on the same device

    with torch.no_grad():
        outputs = finetuned_model(input_ids, labels=input_ids)
        log_prob = -outputs.loss.item()  # Negative loss as confidence score

    return log_prob


In [32]:
for i in generate_sql_queries(output, 5):
  print(get_query_log_prob(i))

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


-0.014908058568835258
-0.02384999394416809
-0.08849175274372101
-0.08051127940416336
-0.2895098328590393


In [33]:
import sqlparse

def is_valid_sql(query):
    """Check if SQL query has valid syntax."""
    try:
        parsed = sqlparse.parse(query)
        return len(parsed) > 0
    except:
        return False


In [35]:
for i in generate_sql_queries(output, 5):
  print(is_valid_sql(i))

True
True
True
True
True


In [36]:
from sentence_transformers import SentenceTransformer, util

# Load a lightweight similarity model
similarity_model = SentenceTransformer("all-MiniLM-L6-v2")

def semantic_similarity(query, original_question):

    embeddings = similarity_model.encode([query, original_question], convert_to_tensor=True)
    score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    return score  # Higher means better match




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [46]:
def compute_confidence(query, original_question, conn=None):

    log_prob = get_query_log_prob(query)
    syntax_valid = is_valid_sql(query)
    similarity = semantic_similarity(query, original_question)
    length_penalty = len(query) / 100  # Normalize penalty

    # Tunable weights
    alpha, beta, delta, epsilon = 1.0, 1.0, 1.0, 10.0

    confidence_score = (
        alpha * log_prob +
        beta * (1 if syntax_valid else -1) -
        delta * length_penalty +
        epsilon * similarity
    )

    return confidence_score

In [47]:
for i in generate_sql_queries(output, 5):
  print(i, compute_confidence(i, context))

SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages' 5.801543090417981
SELECT "Date of Transaction" FROM rec_print_sales WHERE "Date of Transaction" = '10 pages' 6.084007328748703
SELECT SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages' 5.65997760951519
SELECT "Date of Transaction" FROM tables_print_sales WHERE "Date of Transaction" = '10 pages' 5.687733292281628
SELECT "Date of Transaction" FROM SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages' 5.098223378658295


In [43]:
for i in generate_sql_queries(output, 5):
  print(i, semantic_similarity(i, context))

SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages' 0.5736451148986816
SELECT "Date of Transaction" FROM rec_print_sales WHERE "Date of Transaction" = '10 pages' 0.6007857322692871
SELECT SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages' 0.5738469362258911
SELECT "Date of Transaction" FROM tables_print_sales WHERE "Date of Transaction" = '10 pages' 0.5698244571685791
SELECT "Date of Transaction" FROM SELECT "Date of Transaction" FROM table_print_sales WHERE "Date of Transaction" = '10 pages' 0.5647733211517334


# Web app

##server side

In [ ]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("server_Z2VZWQTD5CQBZWLT4VBUAA42-57ADWD323GRPAAAJ")

In [ ]:
@anvil.server.callable
def generate_query(question, context):

  if context:
    prompt = f"""Tables:
    {context}

    Question:
    {question}

    Answer:
    """
  else:
    def_context = """create table rec_print_sales("Date of Transaction" DATE, "Student name" VARCHAR, "Number of print pages" INT, "Number of Xerox copies" INT, "Amount paid" INT);"""
    prompt = f"""Tables:
    {def_context}

    Question:
    {question}

    Answer:
    """
  inputs = tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to('cuda')

  output = tokenizer.decode(finetuned_model.generate(inputs["input_ids"],max_new_tokens=200,)[0],skip_special_tokens=True)

  conn = sqlite3.connect('test.db')
  flag=0
  try:
    df = pd.read_sql(output, conn)
  except:
    flag=1
  if flag==0:
    results=[output, flag, df]
  else:
    results=[output, flag]
  return results

In [ ]:
anvil.server.wait_forever()

#SQL side

## Setup

In [ ]:
import sqlite3

In [ ]:
%load_ext sql

In [ ]:
%%sql
sqlite:///test.db

In [ ]:
%%sql
create table rec_print_sales(Date_of_Transaction date, Student_name varchar, Number_of_print_pages int, Number_of_Xerox_copies int, Amount_paid int);

 * sqlite:///test.db
Done.


[]

In [ ]:
%%sql
INSERT INTO rec_print_sales (Date_of_Transaction, Student_name, Number_of_print_pages, Number_of_Xerox_copies, Amount_paid) VALUES
('2025-03-01', 'Alice Johnson', 15, 3, 50),
('2025-03-02', 'Bob Smith', 30, 5, 100),
('2025-03-03', 'Charlie Brown', 25, 7, 90),
('2025-03-04', 'David White', 40, 2, 120),
('2025-03-05', 'Emma Clark', 10, 4, 45),
('2025-03-06', 'Fiona Harris', 20, 8, 85),
('2025-03-07', 'George Williams', 35, 6, 110),
('2025-03-08', 'Hannah Lee', 50, 10, 160),
('2025-03-09', 'Isaac Moore', 60, 5, 150),
('2025-03-10', 'Jack Taylor', 45, 12, 130),
('2025-03-11', 'Liam King', 20, 3, 60),
('2025-03-12', 'Mia Scott', 30, 5, 95),
('2025-03-13', 'Noah Lewis', 25, 2, 75),
('2025-03-14', 'Olivia Walker', 18, 4, 65),
('2025-03-15', 'Parker Harris', 28, 6, 90),
('2025-03-16', 'Quinn Evans', 36, 7, 105),
('2025-03-17', 'Ryan Mitchell', 22, 3, 70),
('2025-03-18', 'Sophia Adams', 40, 10, 125),
('2025-03-19', 'Thomas Roberts', 48, 8, 140),
('2025-03-20', 'Ursula Cooper', 50, 9, 145),
('2025-03-21', 'Victor Martin', 32, 4, 95),
('2025-03-22', 'Wendy Young', 26, 5, 80),
('2025-03-23', 'Xander Allen', 55, 10, 150),
('2025-03-24', 'Yara Gonzalez', 38, 6, 110),
('2025-03-25', 'Zachary Perez', 41, 3, 115),
('2025-03-26', 'Amelia Garcia', 29, 2, 85),
('2025-03-27', 'Benjamin Kim', 33, 4, 100),
('2025-03-28', 'Chloe Nelson', 24, 7, 85),
('2025-03-29', 'Daniel Clark', 60, 5, 155),
('2025-03-30', 'Ella Turner', 44, 8, 125),
('2025-03-31', 'Finn Walker', 53, 9, 140),
('2025-04-01', 'Grace Harris', 39, 3, 95),
('2025-04-02', 'Henry Lewis', 42, 10, 135),
('2025-04-03', 'Isla Scott', 31, 6, 100),
('2025-04-04', 'James Moore', 27, 5, 85),
('2025-04-05', 'Katherine Brown', 22, 4, 75),
('2025-04-06', 'Leo White', 19, 7, 60),
('2025-04-07', 'Maya Johnson', 36, 6, 110),
('2025-04-08', 'Nina King', 48, 5, 140),
('2025-04-09', 'Oscar Adams', 41, 3, 125),
('2025-04-10', 'Paige Taylor', 28, 9, 95),
('2025-04-11', 'Quincy Robinson', 52, 8, 145),
('2025-04-12', 'Riley Young', 39, 6, 115),
('2025-04-13', 'Sophia Martin', 33, 4, 95),
('2025-04-14', 'Travis Hall', 47, 10, 150),
('2025-04-15', 'Uma Lee', 21, 2, 60),
('2025-04-16', 'Violet Mitchell', 29, 7, 85),
('2025-04-17', 'Willow Roberts', 34, 3, 95),
('2025-04-18', 'Xander Moore', 45, 9, 130),
('2025-04-19', 'Yasmine Green', 38, 4, 105),
('2025-04-20', 'Zane White', 50, 5, 140);

 * sqlite:///test.db
51 rows affected.


[]

In [ ]:
conn = sqlite3.connect('test.db')

## Running SQL

In [ ]:
query = 'SELECT "Date_of_Transaction" FROM rec_print_sales WHERE "Number_of_Xerox_copies" = 3'

In [ ]:
df = pd.read_sql(query, conn)

In [ ]:
df

,Date_of_Transaction
0,2025-03-01
1,2025-03-11
2,2025-03-17
3,2025-03-25
4,2025-04-01
5,2025-04-09
6,2025-04-17
